In [1]:
import sqlite3
import powerlaw
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

ModuleNotFoundError: No module named 'powerlaw'

In [ ]:
# Constants
sqlite3_filename = "FPA_FOD_20170508.sqlite"

In [ ]:
conn = sqlite3.connect(sqlite3_filename)
cur = conn.cursor()
cur.execute('SELECT FIRE_SIZE, STATE FROM fires')
tableData = cur.fetchall()
conn.close()

In [ ]:
# Collect a list of all state abbreviations
states = {}
for i in range(len(tableData)):
    states[tableData[i][1]] = True

In [ ]:
for state in states:

In [ ]:
    # Collect all the data for the current state
    data = []
    minSize = maxSize = tableData[0][0]
    for tableRow in tableData:
        if tableRow[1] == state:
            size = tableRow[0]
            minSize = min(minSize, size)
            maxSize = max(maxSize, size)
            data.append(size)

In [ ]:
    # Get the powerlaw fit for the current state
    results = powerlaw.Fit(data)

In [ ]:
    # Get all the data over the x min result
    overXMinData = []
    for size in data:
        if size > results.xmin:
            overXMinData.append(size)

In [ ]:
    # Print results of the fit
    print("count:", len(data))
    print("overXMin:", len(overXMinData))
    print("min:", minSize)
    print("max:", maxSize)
    print("alpha:", results.alpha)
    print("sigma:", results.sigma)
    print("xmin:", results.xmin)

In [ ]:
    # Compare against other distributions
    # R is the loglikelihood ratio between the two candidate distributions.
    # If R > 0, then the data is more likely to be in the first distribution.
    # If R < 0, then the data is more likely to be in the second distribution.
    # normalized_ratio=True divides R by (sigma*sqrt(n))
    # p is the significance value for the identified direction (i.e., for R > 0 or for R < 0)
    for other in results.supported_distributions:
        R, p = results.distribution_compare('power_law', other, normalized_ratio=True)
        print(other+":")
        print("   R:", R)
        print("   p:", p)